In [77]:
import requests
import pandas as pd
from langdetect import detect
from datetime import datetime, timedelta
from newspaper import Article
import re
import os

path = "Extracted_0415/" # only to 0414
os.mkdir(path)

def to_date(DATE):
    return datetime.strptime(START_DATE, '%Y%m%d%H%M%S')

def from_date(DATE):
    return DATE.strftime("%Y%m%d%H%M%S")

# GLOBALS AND CONSTANTS
query = "conspiracy"
nearby = '"conspiracy%20%coronavirus"'

# Initializations
TODAY = "20200414000000"
START_DATE = "20200101000000"
END_DATE = "20200102000000"

TODAY_datetime = to_date(TODAY)
START_DATE_datetime = to_date(TODAY)
END_DATE_datetime = to_date(TODAY)

while END_DATE_datetime <= TODAY_datetime:
    url = "http://api.gdeltproject.org/api/v2/doc/doc?query=" + query + \
            "&sourcelang:english&MAXRECORDS=250&" \
            "STARTDATETIME=" + START_DATE + "&ENDDATETIME=" + END_DATE + \
            "&sourcecountry:us&near100:" + nearby + "&format=csv"

    data = pd.read_csv(url)

    # consider top 50 articles
    DATA = {"URL":[], "Title":[], "Authors":[], "Text":[], 
            "Stupid_Keywords":[], "Stupid_Summary":[]}
    
    for row in data.head(n=50).iterrows():
        # if it is an english news article, kind of
        if detect(row[1]['Title']) == 'en':
            try:
                if row[1]['URL'] not in DATA['URL']:
                    # removing repeats
                    article = Article(row[1]['URL'])
                    article.download()
                    article.parse()

                    text = article.text
                    text_ = text.replace("\n"," ")
                    text2 = text_.replace("\\","") 
                    if text != "":
                        DATA['URL'].append(row[1]['URL'])
                        DATA['Title'].append(row[1]['Title'])
                        DATA['Authors'].append(article.authors)
                        DATA['Text'].append(text2)
                        article.nlp()
                        DATA['Stupid_Keywords'].append(article.keywords)
                        DATA['Stupid_Summary'].append(article.summary)
            except:
                pass
    
    DATA_tabular = pd.DataFrame.from_dict(DATA)
    DATA_tabular.to_csv(path + 'Scraped_News_' + START_DATE +'.csv')
                
    
    START_DATE_datetime += timedelta(days=1)
    END_DATE_datetime += timedelta(days=1)
    
    START_DATE = from_date(START_DATE_datetime)
    END_DATE = from_date(END_DATE_datetime)